# 1. Should we remove the bias parameter from the fully connected layer or the convolutional layer before the batch normalization? Why?



# 2. Compare the learning rates for LeNet with and without batch normalization.



In [ ]:
class LeNet(d2l.Classifier):  #@save
    """The LeNet-5 model."""
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5, padding=2), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=5), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(120), nn.Sigmoid(),
            nn.LazyLinear(84), nn.Sigmoid(),
            nn.LazyLinear(num_classes))
        
class BNLeNet(d2l.Classifier):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5), nn.LazyBatchNorm2d(),
            nn.Sigmoid(), nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=5), nn.LazyBatchNorm2d(),
            nn.Sigmoid(), nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(), nn.LazyLinear(120), nn.LazyBatchNorm1d(),
            nn.Sigmoid(), nn.LazyLinear(84), nn.LazyBatchNorm1d(),
            nn.Sigmoid(), nn.LazyLinear(num_classes))
        
def stat_model_acc(model, data, trainer):
    model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
    trainer.fit(model, data)
    X,y = next(iter(data.get_dataloader(False)))
    X = X.to('cuda')
    y = y.to('cuda')
    y_hat = model(X) 
    return model.accuracy(y_hat,y).item()

In [ ]:
data = d2l.FashionMNIST(batch_size=128)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
lr_list = [0.001,0.01,0.03,0.1,0.3]
model_list = [BNLeNet,LeNet]
le_acs= []
for lr in lr_list:
    for model_class in model_list:
    


## 2.1 Plot the increase in validation accuracy.



## 2.2 How large can you make the learning rate before the optimization fails in both cases?



# 3. Do we need batch normalization in every layer? Experiment with it.



# 4. Implement a “lite” version of batch normalization that only removes the mean, or alternatively one that only removes the variance. How does it behave?



# 5. Fix the parameters beta and gamma. Observe and analyze the results.



# 6. Can you replace dropout by batch normalization? How does the behavior change?



# 7. Research ideas: think of other normalization transforms that you can apply:



## 7.1 Can you apply the probability integral transform?



## 7.2 Can you use a full-rank covariance estimate? Why should you probably not do that?


## 7.3 Can you use other compact matrix variants (block-diagonal, low-displacement rank, Monarch, etc.)?



## 7.4 Does a sparsification compression act as a regularizer?



## 7.5 Are there other projections (e.g., convex cone, symmetry group-specific transforms) that you can use?